In [47]:
import torch
import torch.nn as nn


class NeuMF(nn.Module):
    def __init__(self, n_users, n_items, latent_mlp, latent_mf, layers):
        super().__init__()
        self.embedding_user_mlp = nn.Embedding(num_embeddings=n_users, embedding_dim=latent_mlp)
        self.embedding_item_mlp = nn.Embedding(num_embeddings=n_items, embedding_dim=latent_mlp)

        self.embedding_user_mf = nn.Embedding(num_embeddings=n_users, embedding_dim=latent_mf)
        self.embedding_item_mf = nn.Embedding(num_embeddings=n_items, embedding_dim=latent_mf)

        self.fc_layers = nn.ModuleList([])
        mlp_out = 0
        for idx, (in_size, out_size) in enumerate(zip(layers[:-1], layers[1:])):
            mlp_out = out_size
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))

        self.affine_output = nn.Linear(latent_mf + mlp_out, 1)
        self.logistic = nn.Sigmoid()

    def forward(self, u, i):
        u_embedding_mlp = self.embedding_user_mlp(u)
        u_embedding_mf = self.embedding_user_mf(u)

        i_embedding_mlp = self.embedding_item_mlp(i)
        i_embedding_mf = self.embedding_item_mf(i)

        mlp_vector = torch.cat([u_embedding_mlp, i_embedding_mlp], dim=-1)
        mf_vector = torch.mul(u_embedding_mf, i_embedding_mf)

        for i in range(len(self.fc_layers)):
            mlp_vector = self.fc_layers[i](mlp_vector)
            mlp_vector = nn.GELU()(mlp_vector)

        vector = torch.cat([mlp_vector, mf_vector], dim=-1)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating


In [48]:
from utils import load_data, split_validate_train, transform_data
import numpy as np

data, test_data, id_index_dict, n_users, n_items = load_data()
train_data, validate_data = split_validate_train(data, scale=100)

train_data = transform_data(train_data)
validate_data = transform_data(validate_data)

epochs = 100
batch_size = 128
layers = np.array([100, 150, 100, 80])
model = NeuMF(n_users=n_users, n_items=n_items, latent_mlp=50, latent_mf=50, layers=layers)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)


Total number of user: 19835
Total number of items: 456722


In [49]:
def train(epoch):
    model.train()
    running_loss = 0
    for idx in range(0, train_data.shape[0], batch_size):
        batch = train_data[idx:idx+batch_size]
        user_ids = torch.tensor(batch[:, 0], dtype=torch.long)
        item_ids = torch.tensor(batch[:, 1], dtype=torch.long)
        ratings = torch.tensor(batch[:, 2], dtype=torch.float)
        preds = model(user_ids, item_ids)
        optimizer.zero_grad()
        loss = criterion(ratings, preds)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'train epoch {epoch + 1}, loss: {running_loss:.3f}')

def test():
    model.eval()
    loss = 0
    for u, i, r in validate_data:
        u = torch.tensor(u, dtype=torch.long)
        i = torch.tensor(i, dtype=torch.long)
        r = torch.tensor(r, dtype=torch.float)

        pred = model(u, i)
        loss += (pred * 100 - r)**2
    
    print(f'validate  rmse: {loss:.3f}')


In [50]:
for epoch in range(epochs):
    train(epoch)
    test()

c:\Users\LEGION\.conda\envs\py38\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([128, 1])) that is different to the input size (torch.Size([128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
